# Logs analysis

In [1]:
%matplotlib inline
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import pandas as pd
import re

## List all logs files

In [2]:
import glob
files = glob.glob('../results/logs/**/events.out.tfevents.*', recursive=True)

len(files), files[0:3]

(180,
 ['../results/logs/kfold=3/kfold-intern=0/batch_size=10/class=RBM/visible_size=702/hidden_size=500/regularization=NoRegularization-0.0/learning_rate=ConstantLearningRate-0.05/sampling_method=CD-1/momentum=1/1542843751.1351352/events.out.tfevents.1542843754.linux',
  '../results/logs/kfold=3/kfold-intern=0/batch_size=10/class=RBM/visible_size=702/hidden_size=500/regularization=NoRegularization-0.0/learning_rate=ConstantLearningRate-0.1/sampling_method=CD-1/momentum=1/1542844084.7186527/events.out.tfevents.1542844087.linux',
  '../results/logs/kfold=3/kfold-intern=0/batch_size=10/class=RBM/visible_size=702/hidden_size=500/regularization=NoRegularization-0.0/learning_rate=ConstantLearningRate-0.01/sampling_method=CD-1/momentum=1/1542843417.7214174/events.out.tfevents.1542843421.linux'])

## Read a file

In [7]:
file = files[0]

def extract_information(file):
    ea = EventAccumulator(file)
    ea.Reload()
    
    scalars = ea.Tags()['scalars']
    first_scalar = ea.Scalars(scalars[0])
    
    dataframes = pd.DataFrame(first_scalar)[['wall_time', 'step']]
    
    for df_name in ea.Tags()['scalars']:
        df = pd.DataFrame(ea.Scalars(df_name))
        series = df['value'].rename(df_name)
        dataframes = pd.concat([dataframes, series], axis=1)
    
    #train = pd.DataFrame(ea.Scalars('measure/evaluate/train'))
    #validation = pd.DataFrame(ea.Scalars('measure/evaluate/validation'))
    
    del ea

    return dataframes

dfs = extract_information(file)
dfs.head()

#train, validation = extract_information(file)
#train.head()

,wall_time,step,measure/reconstruction/error,measure/activation/mean,measure/activation/std,measure/parameters/W/mean,measure/parameters/b_h/mean,measure/parameters/b_v/mean,measure/reconstruction_1/hamming,details/measure/top-1/evaluate-0/train,...,details/measure/expectation/normalized/evaluate-3/train,details/measure/expectation/normalized/evaluate-3/validation,details/measure/expectation/normalized/evaluate-4/train,details/measure/expectation/normalized/evaluate-4/validation,details/measure/expectation/normalized/evaluate-5/train,details/measure/expectation/normalized/evaluate-5/validation,measure/evaluate/expectation/normalized/RMSE_train,measure/evaluate/expectation/normalized/RMSE_validation,measure/evaluate/expectation/normalized/train,measure/evaluate/expectation/normalized/validation
0,1.542844e+09,1,0.124222,5.955204,14.738444,-0.109463,-1.570596,-0.017528,3.057971,0.106719,...,0.014493,0.011842,0.018445,0.017105,0.000000,0.000000,0.130283,0.130370,0.007027,0.005702
1,1.542844e+09,11,0.117921,7.047431,13.660465,-0.110493,-1.671679,-0.057376,2.919631,0.334651,...,0.013175,0.009211,0.014493,0.025000,0.025033,0.022368,0.130052,0.129952,0.010540,0.012061
2,1.542844e+09,21,0.109495,6.374177,9.585950,-0.113326,-1.672066,-0.090734,2.944664,0.293808,...,0.011858,0.013158,0.006588,0.005263,0.010540,0.009211,0.130197,0.130313,0.008344,0.006579
3,1.542844e+09,31,0.102354,6.297760,7.753239,-0.117138,-1.642134,-0.119853,2.836627,0.284585,...,0.010540,0.009211,0.015810,0.025000,0.017128,0.014474,0.130067,0.129981,0.010321,0.011623
4,1.542844e+09,41,0.095702,6.442688,7.877815,-0.121068,-1.637922,-0.144640,2.762846,0.350461,...,0.015810,0.014474,0.025033,0.026316,0.015810,0.007895,0.129706,0.129822,0.015810,0.014035


In [8]:
def extract_columns(file):
    regex = r'(.+?)=(.+?)\/'
    test_str = file.replace('../results/logs/', '')
    matches = re.finditer(regex, test_str, re.MULTILINE)

    columns = {match.groups()[0]: match.groups()[1] for match in matches}
    columns['path'] = test_str
    return columns

def populate_columns(file, dataframe):
    columns = extract_columns(file)
    
    for k, v in columns.items():
        dataframe[k] = v

    return dataframe

In [9]:
def generate_logs_files(files):
    data = []

    for index, file in enumerate(files):
        print(f'{index+1} of {len(files)}')
        information = extract_information(file)
        information = populate_columns(file, information)
        
        data.append(information)
        
    return pd.concat(data)

data = generate_logs_files(files)

1 of 180
2 of 180
3 of 180
4 of 180
5 of 180
6 of 180
7 of 180
8 of 180
9 of 180
10 of 180
11 of 180
12 of 180
13 of 180
14 of 180
15 of 180
16 of 180
17 of 180
18 of 180
19 of 180
20 of 180
21 of 180
22 of 180
23 of 180
24 of 180
25 of 180
26 of 180
27 of 180
28 of 180
29 of 180
30 of 180
31 of 180
32 of 180
33 of 180
34 of 180
35 of 180
36 of 180
37 of 180
38 of 180
39 of 180
40 of 180
41 of 180
42 of 180
43 of 180
44 of 180
45 of 180
46 of 180
47 of 180
48 of 180
49 of 180
50 of 180
51 of 180
52 of 180
53 of 180
54 of 180
55 of 180
56 of 180
57 of 180
58 of 180
59 of 180
60 of 180
61 of 180
62 of 180
63 of 180
64 of 180
65 of 180
66 of 180
67 of 180
68 of 180
69 of 180
70 of 180
71 of 180
72 of 180
73 of 180
74 of 180
75 of 180
76 of 180
77 of 180
78 of 180
79 of 180
80 of 180
81 of 180
82 of 180
83 of 180
84 of 180
85 of 180
86 of 180
87 of 180
88 of 180
89 of 180
90 of 180
91 of 180
92 of 180
93 of 180
94 of 180
95 of 180
96 of 180
97 of 180
98 of 180
99 of 180
100 of 180
101 of 1

In [14]:
data.index.name = 'evaluate'
data.head(5)

,wall_time,step,measure/reconstruction/error,measure/activation/mean,measure/activation/std,measure/parameters/W/mean,measure/parameters/b_h/mean,measure/parameters/b_v/mean,measure/reconstruction_1/hamming,details/measure/top-1/evaluate-0/train,...,kfold-intern,batch_size,class,visible_size,hidden_size,regularization,learning_rate,sampling_method,momentum,path
evaluate,,,,,,,,,,,,,,,,,,,,,
0,1.542844e+09,1,0.124222,5.955204,14.738444,-0.109463,-1.570596,-0.017528,3.057971,0.106719,...,0,10,RBM,702,500,NoRegularization-0.0,ConstantLearningRate-0.05,CD-1,1,kfold=3/kfold-intern=0/batch_size=10/class=RBM...
1,1.542844e+09,11,0.117921,7.047431,13.660465,-0.110493,-1.671679,-0.057376,2.919631,0.334651,...,0,10,RBM,702,500,NoRegularization-0.0,ConstantLearningRate-0.05,CD-1,1,kfold=3/kfold-intern=0/batch_size=10/class=RBM...
2,1.542844e+09,21,0.109495,6.374177,9.585950,-0.113326,-1.672066,-0.090734,2.944664,0.293808,...,0,10,RBM,702,500,NoRegularization-0.0,ConstantLearningRate-0.05,CD-1,1,kfold=3/kfold-intern=0/batch_size=10/class=RBM...
3,1.542844e+09,31,0.102354,6.297760,7.753239,-0.117138,-1.642134,-0.119853,2.836627,0.284585,...,0,10,RBM,702,500,NoRegularization-0.0,ConstantLearningRate-0.05,CD-1,1,kfold=3/kfold-intern=0/batch_size=10/class=RBM...
4,1.542844e+09,41,0.095702,6.442688,7.877815,-0.121068,-1.637922,-0.144640,2.762846,0.350461,...,0,10,RBM,702,500,NoRegularization-0.0,ConstantLearningRate-0.05,CD-1,1,kfold=3/kfold-intern=0/batch_size=10/class=RBM...


In [15]:
data.to_csv('extracted_log.csv')